#Machine Learnt Patterns in Rhodium-Catalysed Asymmteric Michael Addition Using Chiral Diene Ligands

Welcome to the Jupyter Notebook version of the python code for training
regression models, random forests, and neural networks to find relationships between diene ligands
and the asymmetric Michael Addition they perform

The first part of using python code is installing and calling the appropriate "libraries".
Libraries are prewritten code that perform lots of work for us.
This means less writing code from scratch.
Libraries are maintained by different groups and can be downloaded as and when needed for your project.
Different projects can use "Environments" so that you only use the libraries needed for each project.

Here we are using Pandas, numpy, math, keras, sys, sklearn (aka sci-kit learn) and tensorflow.

Balancedkfold is  our own bit of code in another python file, balancedkfold.py

In [1]:
import pandas as pd
import numpy as np
import math
import keras
from sys import exit
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout

import tensorflow as tf
from sklearn.model_selection import train_test_split
import balancedkfold
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.metrics import mean_squared_error, r2_score


With the libraries loaded - or just parts of them - we now move into the main code.

We define a few functions first that we will use in the main routine.


We define a function called 'choose_model' that will use our best parameters held in the
dictionary 'best_params' for running our model.

Here we have 'commented out' - turned off - all models but Random Forest Regression.

If there are no paramters given then the parameters will be searched for to give the best fit.

In [2]:


def choose_model(best_params):
    if best_params == None:
        # return LinearRegression()
        return RandomForestRegressor()
        # return GradientBoostingRegressor()
        # return SVR()

    else:
        # return LinearRegression()
        return RandomForestRegressor(n_estimators=best_params["n_estimators"], max_depth=best_params["max_depth"], min_samples_leaf=best_params['min_samples_leaf'], min_samples_split=best_params['min_samples_split'])
        # return GradientBoostingRegressor(loss = best_params['loss'], learning_rate=best_params['learning_rate'],n_estimators=best_params["n_estimators"], max_depth=best_params["max_depth"],min_samples_leaf=best_params['min_samples_leaf'], min_samples_split=best_params['min_samples_split'])
        # return SVR()


Here the function is choosing the data set to be used. Again those we are not using are commented out.

In [3]:

def choose_dataset():
    return 'Cyclic'
    # return 'Acyclic'
    # return 'Combined'
    # return 'Cyclohexenone'


This function is for tuning the hyper parameters, if we call the function to do so.

In [4]:

def hyperparam_tune(X, y, model):
    print(str(model))
    if str(model) == 'RandomForestRegressor()':
        hyperP = dict(n_estimators=[100, 300, 500, 800], max_depth=[None, 5, 8, 15, 25, 30],
                      min_samples_split=[2, 5, 10, 15, 100],
                      min_samples_leaf=[1, 2, 5, 10])

    elif str(model) == 'GradientBoostingRegressor()':
        hyperP = dict(loss=['ls'], learning_rate=[0.1, 0.2, 0.3],
                      n_estimators=[100, 300, 500, 800], max_depth=[None, 5, 8, 15, 25, 30],
                      min_samples_split=[2],
                      min_samples_leaf=[1, 2])

    elif str(model) == 'SVR()':
        hyperP = dict(kernel=['linear', 'rbf', 'poly', 'sigmoid'], gamma=['scale', 'auto'], C=[0.1, 1, 5, 10],
                      epsilon=[0.001, 0.01, 0.1, 1, 5])


    gridF = GridSearchCV(model, hyperP, cv=3, verbose=1, n_jobs=-1)
    bestP = gridF.fit(X, y)
    print(bestP.best_params_)
    return bestP.best_params_

Next we define the random seeds.

Computers are pseudo random, so we need to initialise our random number generators

In [5]:

random_seeds = np.random.random_integers(0, high=1000, size=30)
print(random_seeds)

[831 875 107 124 806 526 107 768 212 695 689 575 822 576 776  58 293 262
 663 277  83 923 181 939 454 932 795 628 704  33]


/var/folders/t7/mm1tdtr101z3xh07kxn136xc0000gn/T/ipykernel_17814/299116365.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 1000 + 1) instead
  random_seeds = np.random.random_integers(0, high=1000, size=30)


This is the 'list' of descriptor names. Basically the headings from our CSV file.

The CSV file is read in using the pandas read_csv function, and the columns in our file
that do not correspond to the descriptors we have defined are removed

In [6]:

descriptors = ['LVR1', 'LVR2', 'LVR3', 'LVR4', 'LVR5', 'LVR6', 'LVR7', 'VB', 'ER1', 'ER2', 'ER3', 'ER4', 'ER5', 'ER6',
               'ER7', 'SStoutR1', 'SStoutR2', 'SStoutR3', 'SStoutR4', '%top']

data = pd.read_csv(choose_dataset()+'.csv')

data = data.filter(descriptors)

With the data read in we also remove those rows containing any erroneous values


In [7]:
#remove erroneous data
data = data.dropna(axis=0)
print(data)

           LVR1  LVR2       LVR3       LVR4       LVR5       LVR6       LVR7  \
0     94.184290   0.0   0.000000  94.184290   0.000000   0.000000  20.255985   
1     94.184290   0.0   0.000000  94.184290   0.000000   0.000000  20.255985   
2     53.529726   0.0   0.000000   0.000000  21.574215   0.000000  76.074168   
3     36.233741   0.0   0.000000  73.462170  21.574215   0.000000  76.074168   
4     56.166185   0.0   0.000000  76.888305  21.574215   0.000000  76.074168   
..          ...   ...        ...        ...        ...        ...        ...   
159    0.000000   0.0  95.697682   0.000000   0.000000  95.697682  20.255985   
160  168.732323   0.0   0.000000   0.000000   0.000000   0.000000  32.691970   
161   94.184290   0.0   0.000000  94.184290   0.000000   0.000000  20.255985   
162   64.956412   0.0   0.000000  21.574215   0.000000   0.000000  84.579926   
163   64.956412   0.0   0.000000  21.574215   0.000000   0.000000  84.579926   

             VB   ER1  ER2   ER3   ER4 

We split our data into target  'Y', and descriptors 'X'.

In [8]:

X = data.drop(['%top'], axis = 1)
X = RobustScaler().fit_transform(np.array(X))
y = data['%top']
print(X)
print(y)

[[ 0.18363981  0.          0.         ...  1.          0.
   0.        ]
 [ 0.18363981  0.          0.         ...  0.          0.
   0.        ]
 [-0.24800929  0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.18363981  0.          0.         ...  0.          0.
   0.        ]
 [-0.12668666  0.          0.         ...  0.          0.
   0.        ]
 [-0.12668666  0.          0.         ...  0.          0.
   0.        ]]
0       6.000000
1       2.333333
2      79.000000
3      97.500000
4      91.000000
         ...    
159    91.000000
160     4.000000
161     2.000000
162     0.300000
163     0.300000
Name: %top, Length: 164, dtype: float64


Here we have predefined our best parameters as a 'dictionary'.
The parameter searching function usage is commented out.

We also define containers for storing the results of our training.

In [9]:

#best_params = hyperparam_tune(X, y, choose_model(best_params=None)) #hyperparameter tuning completed on whole subset
best_params = {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 500}
r2_cv_scores = []
rmse_cv_scores = []
r2_val_scores = []
rmse_val_scores = []


With the models now all set up and ready to run we can start the training

We loop over our random seeds, as we want to try different starting positions
in the search space for fitting the model.

Each time the data is split randomly into test and validation data to prevent overfitting.

K fold crossvalidation fitting is also performed to further ensure no over fitting.

In [10]:

for i in range(len(random_seeds)):
    # split into training and validation sets, 9:1
    X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                  test_size=0.1, random_state=random_seeds[i])
    X_train = np.array(X_train).astype('float64')
    X_val = np.array(X_val).astype('float64')
    y_train = np.array(y_train).astype('float64')
    y_val = np.array(y_val).astype('float64')

    # 5 fold CV on training set, repeated 3 times
    for j in range(3):
        kfold = balancedkfold.BalancedKFold(5, verbose=False)  # Kohavi, 1995, stratified KFold
        for train, test in kfold.split(X_train, y_train):
            model = choose_model(best_params)
            model.fit(X_train[train], y_train[train])
            predictions = model.predict(X_train[test]).reshape(1, -1)[0]

            r2 = r2_score(y_train[test], predictions)
            rmse = math.sqrt(mean_squared_error(predictions, y_train[test]))
            r2_cv_scores.append(r2)
            rmse_cv_scores.append(rmse)


    # predict on validaiton set
    model = choose_model(best_params)
    model.fit(X_train, y_train)

    predictions = model.predict(X_val)
    r2 = r2_score(y_val, predictions)
    rmse = math.sqrt(mean_squared_error(predictions, y_val))
    r2_val_scores.append(r2)
    rmse_val_scores.append(rmse)

KeyboardInterrupt: 

The final model results are then return to the command line.

In [ ]:

print('Model:',  model)
print('Data Subset: ',  choose_dataset())
print('Random Seeds: ', random_seeds, '\n')
print('Num CV Scores: ', len(r2_cv_scores))
print('CV R2 Mean: ', np.mean(np.array(r2_cv_scores)), '+/-', np.std(np.array(r2_cv_scores)))
print('CV RMSE Mean %: ', np.mean(np.array(rmse_cv_scores)), '+/-', np.std(np.array(rmse_cv_scores)), '\n')
print('Num Val Scores: ', len(r2_val_scores))
print(r2_val_scores)
print('Val r2 Mean: ', np.mean(np.array(r2_val_scores)), '+/-', np.std(np.array(r2_val_scores)))
print('Val RMSE Mean %: ', np.mean(np.array(rmse_val_scores)), '+/-', np.std(np.array(rmse_val_scores)))
